In [1]:
import networkit as nk

import os
import time

In [2]:
import numpy as np

In [3]:
DATA = "all_ranks_graph"
ORIGINAL_NETWORK_NAME = "output_graph.csv"
TEMP_NETWORKS_FOLDER = "temp"

BASE = os.path.join(os.path.abspath(os.curdir), DATA)
PATH_TO_NETWORK_ORIGINAL = os.path.join(BASE, ORIGINAL_NETWORK_NAME)
PATH_TO_NETWORK_PROCESSED = os.path.join(BASE, "processed", ORIGINAL_NETWORK_NAME.replace(".csv", ""))

In [4]:
edgeListReader = nk.graphio.EdgeListReader(',', 0, continuous=False, directed=False)
G = edgeListReader.read(PATH_TO_NETWORK_ORIGINAL)
original_nodes = G.numberOfNodes()

G.removeMultiEdges()
G.compactEdges()
G.removeSelfLoops()
G.compactEdges()

print("Nodes: ", original_nodes, " Edges: ", G.numberOfEdges())
print("Consistent?: ", G.checkConsistency())

Nodes:  313673  Edges:  1885386
Consistent?:  True


In [5]:
components = nk.components.ConnectedComponents(G).run()
connected_components = components.numberOfComponents()
if connected_components != 1:
    print("More than 1 component, extracting largest one.")
    G = components.extractLargestConnectedComponent(G)
    print("AVG Clustering Coefficient: ", np.mean(nk.centrality.LocalClusteringCoefficient(G).run().scores()))

More than 1 component, extracting largest one.
AVG Clustering Coefficient:  0.8895137729153759


In [6]:
l_cliques = nk.clique.MaximalCliques(G, maximumOnly=False).run().getCliques()

In [58]:
__clique_tuple = [tuple(clique) for clique in l_cliques]
clique_dictionary = dict(zip(__clique_tuple, range(len(__clique_tuple))))

In [ ]:
nodes_to_clique = {}

# Around 15min single core (for all_ranks)
progress = 0
max_progress = G.numberOfNodes()
start = time.time()
for node in G.iterNodes():
    if progress % 10000 == 0:
        elapsed = time.time() - start
        start = time.time()
        print("Completed {0}/{1}".format(progress, max_progress))
        print("Elapsed " + str(elapsed))
        
    __collected = []
    for clique in __clique_tuple:
        if node in clique:
            __collected.append(clique_dictionary[clique])
    nodes_to_clique[node] = __collected
    progress += 1

In [ ]:
G.indexEdges()
g_no_cliques = nk.Graph(n=len(__clique_tuple)) # Uses all defaults at false

__nodes = []
def callback_iterator(node_id_src, node_id_dst, edge_weight, edge_id):
    __nodes.append(node_id_dst)

In [ ]:
def get_clique_super_node(mini_node_id, different_then=-1):
    for clique in clique_dictionary.keys():
        if mini_node_id in clique:
            if clique_dictionary[clique] != different_then:
                return clique_dictionary[clique]
    print("ERROR!")

def condensation_reach(g_no_cliques, clique, src_mini_node_id_intermediate, src_super_node_id):
    valid_neighbours = []

    G.forEdgesOf(src_mini_node_id_intermediate, callback_iterator)
    src_super_node_id_intermediate = get_clique_super_node(src_mini_node_id_intermediate)
    
    for potential_dst_mini_node_id in __nodes:
        if potential_dst_mini_node_id not in clique: # Not in the original clique, goes from potential to actual
            all_cliques_of_dst = nodes_to_clique[potential_dst_mini_node_id] # List all cliques that a node belongs to
            for dst_super_node_id in all_cliques_of_dst: # For every clique that node belongs to, add a connection to it from the original clique
                if dst_super_node_id != src_super_node_id_intermediate:
                    valid_neighbours.append(potential_dst_mini_node_id)
                    g_no_cliques.addEdge(src_super_node_id, dst_super_node_id, checkMultiEdge=True)     
    return valid_neighbours

# No more than 20min on single core (for all_ranks)
progress = 0
max_progress = len(l_cliques)
elapsed = 0
start = time.time()
max_reach = 1
for clique in l_cliques:
    if progress % 1000 == 0:
        elapsed = time.time() - start
        start = time.time()
        print("Completed {0}/{1}".format(progress, max_progress))
        print("Elapsed " + str(elapsed))
    
    for src_mini_node_id in clique:
        valid_neighbours = [src_mini_node_id]
        src_super_node_id = get_clique_super_node(src_mini_node_id)
        for depth in range(max_reach):
            for intermediate_node in valid_neighbours:
                valid_neighbours = condensation_reach(g_no_cliques, clique, intermediate_node, src_super_node_id) 
                __nodes = [] # Will be used in the callback in condesensation_reach
                
    progress += 1

In [ ]:
# Original code used in the report
# No more than 20min on single core (for all_ranks)
progress = 0
max_progress = len(l_cliques)
elapsed = 0
start = time.time()
for clique in l_cliques:
    if progress % 1000 == 0:
        elapsed = time.time() - start
        start = time.time()
        print("Completed {0}/{1}".format(progress, max_progress))
        print("Elapsed " + str(elapsed))
        
    for src_mini_node_id in clique:
        G.forEdgesOf(src_mini_node_id, callback_iterator)
        src_super_node_id = get_clique_super_node(src_mini_node_id)
        
        for potential_dst_mini_node_id in __nodes:
            if potential_dst_mini_node_id not in clique: # Not in clique of src, potential -> actual
                all_cliques_of_dst = nodes_to_clique[potential_dst_mini_node_id]
                for dst_super_node_id in all_cliques_of_dst:
                    if dst_super_node_id != src_super_node_id:
                        g_no_cliques.addEdge(src_super_node_id, dst_super_node_id, checkMultiEdge=True)                
        __nodes = []
    progress += 1

In [95]:
assert g_no_cliques.numberOfNodes() == len(l_cliques)

In [96]:
writergml = nk.graphio.GraphMLWriter()
writer_edgelist = nk.graphio.EdgeListWriter(separator=",", firstNode=0, bothDirections=False)

In [97]:
writer_edgelist.write(g_no_cliques, PATH_TO_NETWORK_PROCESSED + "_no_cliques_self_loops.csv")
writergml.write(g_no_cliques, os.path.join(PATH_TO_NETWORK_PROCESSED + "_no_cliques_self_loops.graphml"))

In [98]:
g_no_cliques.removeSelfLoops()
g_no_cliques.compactEdges()

writer_edgelist.write(g_no_cliques, PATH_TO_NETWORK_PROCESSED + "_no_cliques.csv")
writergml.write(g_no_cliques, os.path.join(PATH_TO_NETWORK_PROCESSED + "_no_cliques.graphml"))